# Scrape by Instructors

This notebook takes in a data frame of instructors' names and filter classes based on them

In [1]:
#Load selinimum and automatically install the Chrome Driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

from selenium.webdriver.chrome.service import  Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from getpass import getpass
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from Login import login_to_SIS

In [2]:
# FireFox Driver
driver = webdriver.Firefox()
time.sleep(5)

login_to_SIS(driver)

Please enter your MSU email: 


 danganh1


Please enter your password: 


 ········


Please enter your SMS code: 


 780947


In [3]:
try:
    element = driver.find_element(By.ID, "win0groupletPTNUI_LAND_REC_GROUPLET$1") # Click on Classes
    element.click()
    time.sleep(5)
except: # In case the website want to login again
    element = driver.find_element(By.ID, 'loginUrl1') # Click on Login
    element.click()
    time.sleep(5)
    
    element = driver.find_element(By.ID, "win0groupletPTNUI_LAND_REC_GROUPLET$1")  # Click on Classes
    element.click()
    time.sleep(5)

try:
    element = driver.find_element(By.ID, 'SCC_LO_FL_WRK_SCC_VIEW_BTN$2')  # Click on Class Search & Enroll
    element.click()
    time.sleep(5)
except:
    pass # If there's no Class Search & Enroll, proceed

In [4]:
body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
cells = soup.find_all("tr", class_="ps_grid-row psc_rowact")
semesters_dict = dict()
for cell in cells: # Find each semester's ID
    semester = cell.find("a", class_="ps-link")
    semesters_dict[semester.text] = semester.get("href")

previous_semesters = []
current_semesters = []
for key, value in semesters_dict.items():
    if "CSTRMPRV" in value: # Get all previous semesters in a list
        previous_semesters.append(key)
    else: # Get all current (following) semesters in another list
        current_semesters.append(key)

previous_semesters.reverse()
semesters_list = previous_semesters + current_semesters

In [5]:
df = pd.read_csv('CMSE_Names.csv')

In [6]:
def search_by_instructor(instructor, semester):
    wait = WebDriverWait(driver, 15)
    try:
        cur_page = driver.find_element(By.ID, "TERM_VAL_TBL_DESCR").text
    except:
        cur_page = None

    if not cur_page:
        driver.execute_script(semesters_dict[semester])
        time.sleep(2)
    elif cur_page != semester.replace('Semester ', ''):
        change_semester = "javascript:submitAction_win9(document.win9,'DERIVED_SSR_FL_SSR_CHANGE_BTN');"
        driver.execute_script(change_semester);
        time.sleep(3)
        driver.switch_to.frame(0)
        if semester in previous_semesters:
            element = driver.find_element(By.ID, "DERIVED_SSR_FL_SSR_CSTRMPRV_GRP")
            element.click()
            time.sleep(1)
        element = driver.find_element(By.LINK_TEXT, semester)
        driver.execute_script("arguments[0].click();", element)
        time.sleep(5)    
        driver.switch_to.default_content()
        
    element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_CLEAR_FLDS_PB') # Click New Search
    driver.execute_script("arguments[0].click();", element)
    time.sleep(3)
    
    element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_GROUP3')  # Click the Instructors filter
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)
    try:
        element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_MSU_LOAD_INSTRUCTS') # Click Load Instructors
        driver.execute_script("arguments[0].click();", element)
        time.sleep(5)
    except:
        pass
    
    element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_LAST_NAME')  # Find the input field
    element.send_keys(instructor) 
    time.sleep(1)

    try: 
        element = driver.find_element(By.ID, "win9divMSU_CLSRCH_WRK2_GROUPBOX15")
        first = element.find_element(By.CSS_SELECTOR, "tr:not([style])") # Find the first check box
    except:
        return None
    
    driver.execute_script("arguments[0].click();", first.find_element(By.CLASS_NAME, "ps-checkbox")) # Click the first check box
    
    element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_SEARCH_BTN') # Hit Search
    driver.execute_script("arguments[0].click();", element)
    wait.until(EC.element_to_be_clickable((By.ID, "MSU_CLSRCH_I_DV$selm$0$$0")))
    
    i = 0
    while True:
        try: # Find a checkbox based on index
            element = driver.find_element(By.ID, f"MSU_CLSRCH_I_DV$0_row_{i}")
        except: # If there's no more checkbox, break out of loop
            break 
        name = element.find_element(By.ID, f"MSU_CLSRCH_WRK2_INSTR_NAME${i}").text
        # If the checkbox is the instructor, don't click
        if name == instructor.replace(' ', ''): 
            i += 1
            continue
        # Else, click to unselect the checkbox
        checkbox = element.find_element(By.ID, f"MSU_CLSRCH_I_DV$selm${i}$$0")
        driver.execute_script("arguments[0].click();", checkbox)
        i += 1

    try:
        element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_APPLY_PB$186$') # Click Filter Results
        driver.execute_script("arguments[0].click();", element)
        time.sleep(5)
    except:
        element = driver.find_element(By.ID, '#ICOK')
        element.click()
        return None

    classes = set()
    for i in range(50):
        try: # Find the class that the instructor taught
            element = driver.find_element(By.ID, f"DESCR100$0_row_{i}") 
        except: 
            break
        class_name = element.find_element(By.XPATH, f".//div[@id='win9divMSU_RSLT_NAV_WK_HTMLAREA${i}']").text
        classes.add(class_name) # Add the class name to the set
        
    return classes

In [8]:
def search_multiple_semesters(df, start_semester=None, end_semester=None):
    if not start_semester:
        start_semester = semesters_list[0]
    if not end_semester:
        end_semester = semesters_list[-1]
    start_index = semesters_list.index(start_semester)
    end_index = semesters_list.index(end_semester)
    sublist = semesters_list[start_index:end_index+1]
    for semester in sublist:
        df[f"{semester}"]= df["CMSE Faculty Name"].apply(
            lambda name: search_by_instructor(name, semester))

search_multiple_semesters(df, start_semester="Fall Semester 2023", end_semester="Spring Semester 2024")

In [9]:
df

,CMSE Faculty NetID,CMSE Faculty Name,Fall Semester 2023,Spring Semester 2024
0,aalessio,"Alessio, Adam","{BME 892: Biomedical Engineering Seminar, CMSE...",None
1,alexrd,"Dickson, Alexander",None,{CMSE 201: Computational Modeling and Data Ana...
2,ayadav,"Yadav, Aman",None,None
3,browned,"Brown, Edward",None,None
4,caball14,"Caballero, Danny",{PHY 415: Methods of Theoretical Physics},{CMSE 891: Independent Study in Computational ...
5,chitwoo9,"Chitwood, Daniel",{BMB 841: Foundation in Computational and Plan...,None
6,christli,"Christlieb, Andrew",{MTH 890: Readings in Mathematics Approval Req...,{CMSE 891: Independent Study in Computational ...
7,gaotong,"Gao, Tong",{ME 490: Independent Study in Mechanical Engin...,{ME 490: Independent Study in Mechanical Engin...
8,hcy,"Yu, Hui-Chia",{CMSE 821: Numerical Methods for Differential ...,"{MSE 260: Electronic, Magnetic, Thermal, and O..."
9,huangl3,"Huang, Longxiu",{CMSE 890: Selected Topics in Computational Ma...,{CMSE 314: Matrix Algebra with Computational A...


In [ ]:
df.to_csv('CMSE_Names.csv')